<a href="https://colab.research.google.com/github/Braelin2/ECGR5105HW5/blob/main/ECGR5105_HW5_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch as torch
import tensorflow as tf
import torch.optim as optim


from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/My Drive/ECGR5105/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))

In [ ]:
def model(t_u, w5, w4, w3, w2, w1, b):
    return (w5*t_u**5) + (w4*t_u**4) + (w3*t_u**3) + (w2*t_u**2) + (w1*t_u) + b

In [ ]:
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

In [ ]:
def training_loop(n_epochs, optimizer, params, t_u_train, t_u_test, t_c_train, t_c_test):
  for epoch in range(1, n_epochs + 1):
    if params.grad is not None:
      params.grad.zero_()

    t_p_train = model(t_u_train, *params)
    train_loss = loss_fn(t_p_train.transpose(0,1) , t_c_train)

    t_p_test = model(t_u_test, *params)
    test_loss = loss_fn(t_p_test.transpose(0,1), t_c_test)

    optimizer.zero_grad()
    train_loss.backward()
    test_loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
     print('Epoch %d, Training Loss %d, Testing Loss %f' % (epoch, float(train_loss), float(test_loss)))
  return params

In [ ]:
from sklearn.model_selection import train_test_split

np.random.seed(0)
training, testing = train_test_split(dataset, train_size = 0.8, test_size = 0.2, random_state = 100)

Y_train = training.pop("price")
Y_test = testing.pop("price")

varlist =  ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

In [ ]:
from sklearn.preprocessing import StandardScaler
standScaler = StandardScaler()

X_train_temp = training
X_train_temp[varlist] = standScaler.fit_transform(training[varlist])

X_test_temp = testing
X_test_temp[varlist] = standScaler.fit_transform(testing[varlist])

num_train = len(X_train_temp)
num_test = len(X_test_temp)

X_train_temp1 = np.c_[np.ones((num_train, 1)), X_train_temp[varlist]]
X_test_temp1 = np.c_[np.ones((num_test, 1)), X_test_temp[varlist]]

X_train_tensor = torch.tensor(X_train_temp1)
X_test_tensor = torch.tensor(X_test_temp1)

Y_train_tensor = torch.tensor(Y_train.values)
Y_test_tensor = torch.tensor(Y_test.values)

Y_train_normal = (Y_train_tensor - Y_train_tensor.float().mean())/Y_train.std()
Y_test_normal = (Y_test_tensor - Y_test_tensor.float().mean())/Y_test.std()

In [ ]:
# SGD Optimizer:
# Learning Rate 0.00001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-5
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 22809144445701, Testing Loss 21404015541633.937500
Epoch 1000, Training Loss 21378466024792, Testing Loss 19890195314928.925781
Epoch 1500, Training Loss 20435487381935, Testing Loss 18903799955535.957031
Epoch 2000, Training Loss 19631986636813, Testing Loss 18102701316564.980469
Epoch 2500, Training Loss 18883526321979, Testing Loss 17385131306782.707031
Epoch 3000, Training Loss 18171079428626, Testing Loss 16718094835498.162109
Epoch 3500, Training Loss 17490686522498, Testing Loss 16089323554483.707031
Epoch 4000, Training Loss 16841269899588, Testing Loss 15493303723187.179688
Epoch 4500, Training Loss 16221986124445, Testing Loss 14926956409258.468750
Epoch 5000, Training Loss 15631819485096, Testing Loss 14388197279743.562500


tensor([-34429.5430, 138333.7656,  89859.0547, 458742.9375, 102777.7422,
        745468.9375], requires_grad=True)

In [ ]:
# SGD Optimizer:
# Learning Rate 0.000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-6
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 25588593254810, Testing Loss 24164663383531.261719
Epoch 1000, Training Loss 25136998359850, Testing Loss 23730594860869.046875
Epoch 1500, Training Loss 24732587618588, Testing Loss 23337909024454.121094
Epoch 2000, Training Loss 24369405956576, Testing Loss 22981677805386.851562
Epoch 2500, Training Loss 24042251453523, Testing Loss 22657577656720.839844
Epoch 3000, Training Loss 23746574357713, Testing Loss 22361808779132.839844
Epoch 3500, Training Loss 23478398771497, Testing Loss 22091033947162.433594
Epoch 4000, Training Loss 23234249662684, Testing Loss 21842320555873.164062
Epoch 4500, Training Loss 23011091247504, Testing Loss 21613091855962.484375
Epoch 5000, Training Loss 22806269587144, Testing Loss 21401081546176.207031


tensor([-22909.2617, 115913.3359,  59107.9883,  77523.8203,  21004.1074,
         89170.0078], requires_grad=True)

In [ ]:
# SGD Optimizer:
# Learning Rate 0.0000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-7
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26029840326639, Testing Loss 24609494407248.542969
Epoch 1000, Training Loss 25986515783321, Testing Loss 24545527239121.125000
Epoch 1500, Training Loss 25935400673159, Testing Loss 24495298385906.613281
Epoch 2000, Training Loss 25884076314267, Testing Loss 24446409789116.261719
Epoch 2500, Training Loss 25833277775813, Testing Loss 24398086248521.886719
Epoch 3000, Training Loss 25783051494813, Testing Loss 24350266621197.656250
Epoch 3500, Training Loss 25733393731665, Testing Loss 24302940986485.992188
Epoch 4000, Training Loss 25684297059035, Testing Loss 24256102709461.597656
Epoch 4500, Training Loss 25635754113291, Testing Loss 24209745737585.027344
Epoch 5000, Training Loss 25587757195527, Testing Loss 24163863681118.867188


tensor([ 1857.8185, 16858.9121,  8209.5625,  8944.8672,  2526.6177,  9280.1904],
       requires_grad=True)

In [ ]:
# SGD Optimizer:
# Learning Rate 0.000000001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-8
optimizer = optim.SGD([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26283880008515, Testing Loss 24977338426172.429688
Epoch 1000, Training Loss 26174017064096, Testing Loss 24859944198067.160156
Epoch 1500, Training Loss 26112375866292, Testing Loss 24782442210813.625000
Epoch 2000, Training Loss 26078037364895, Testing Loss 24729959506744.035156
Epoch 2500, Training Loss 26058888524340, Testing Loss 24693356537177.320312
Epoch 3000, Training Loss 26047974447300, Testing Loss 24666970282847.609375
Epoch 3500, Training Loss 26041347808254, Testing Loss 24647256653603.339844
Epoch 4000, Training Loss 26036804694554, Testing Loss 24631971293539.429688
Epoch 4500, Training Loss 26033144177752, Testing Loss 24619674340685.718750
Epoch 5000, Training Loss 26029735860167, Testing Loss 24609427952609.796875


tensor([4909.7744, 2759.7720, 1077.9115,  963.7599,  272.8635,  936.1778],
       requires_grad=True)

In [ ]:
# ADAM Optimizer
# Learning Rate 0.1
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26465369593497, Testing Loss 25147493382079.199219
Epoch 1000, Training Loss 26453517368410, Testing Loss 25136531363643.414062
Epoch 1500, Training Loss 26441825489182, Testing Loss 25125679166617.507812
Epoch 2000, Training Loss 26430285712800, Testing Loss 25114929634774.218750
Epoch 2500, Training Loss 26418891068480, Testing Loss 25104276675231.632812
Epoch 3000, Training Loss 26407635913034, Testing Loss 25093715335461.871094
Epoch 3500, Training Loss 26396515669544, Testing Loss 25083241567224.332031
Epoch 4000, Training Loss 26385527082944, Testing Loss 25072852499594.910156
Epoch 4500, Training Loss 26374667308277, Testing Loss 25062545598808.515625
Epoch 5000, Training Loss 26363934604568, Testing Loss 25052319317649.429688


tensor([493.3556, 496.0881, 497.5134, 499.7987, 499.8281, 499.8968],
       requires_grad=True)

In [ ]:
# ADAM Optimizer
# Learning Rate 0.001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-3
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26477246390584, Testing Loss 25158439755920.660156
Epoch 1000, Training Loss 26477125340148, Testing Loss 25158328378735.695312
Epoch 1500, Training Loss 26477004302495, Testing Loss 25158217009544.128906
Epoch 2000, Training Loss 26476883277448, Testing Loss 25158105648158.726562
Epoch 2500, Training Loss 26476762265577, Testing Loss 25157994295133.332031
Epoch 3000, Training Loss 26476641266314, Testing Loss 25157882949915.765625
Epoch 3500, Training Loss 26476520279694, Testing Loss 25157771612536.527344
Epoch 4000, Training Loss 26476399305644, Testing Loss 25157660282932.386719
Epoch 4500, Training Loss 26476278344198, Testing Loss 25157548961132.738281
Epoch 5000, Training Loss 26476157395360, Testing Loss 25157437647142.027344


tensor([5.9997, 5.9997, 5.9997, 5.9997, 5.9997, 4.9998], requires_grad=True)

In [ ]:
# ADAM Optimizer
# Learning Rate 0.0001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26477355130590, Testing Loss 25158539803249.910156
Epoch 1000, Training Loss 26477343021111, Testing Loss 25158528661970.796875
Epoch 1500, Training Loss 26477330911753, Testing Loss 25158517520765.394531
Epoch 2000, Training Loss 26477318802522, Testing Loss 25158506379638.089844
Epoch 2500, Training Loss 26477306693417, Testing Loss 25158495238588.894531
Epoch 3000, Training Loss 26477294584509, Testing Loss 25158484097687.105469
Epoch 3500, Training Loss 26477282475727, Testing Loss 25158472956863.414062
Epoch 4000, Training Loss 26477270367072, Testing Loss 25158461816117.828125
Epoch 4500, Training Loss 26477258258543, Testing Loss 25158450675450.351562
Epoch 5000, Training Loss 26477246150140, Testing Loss 25158439534860.980469


tensor([1.5001, 1.5001, 1.5001, 1.5001, 1.5001, 0.5000], requires_grad=True)

In [ ]:
# ADAM Optimizer
# Learning Rate 0.00001
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad = True)
learning_rate = 1e-5
optimizer = optim.Adam([params], lr = learning_rate)

training_loop(n_epochs = 5000, optimizer = optimizer, params = params, t_u_train = X_train_tensor, t_u_test = X_test_tensor, t_c_train = Y_train_tensor, t_c_test = Y_test_tensor)

Epoch 500, Training Loss 26477366006041, Testing Loss 25158549809136.164062
Epoch 1000, Training Loss 26477364793691, Testing Loss 25158548693726.160156
Epoch 1500, Training Loss 26477363581343, Testing Loss 25158547578318.160156
Epoch 2000, Training Loss 26477362368993, Testing Loss 25158546462907.160156
Epoch 2500, Training Loss 26477361156643, Testing Loss 25158545347496.390625
Epoch 3000, Training Loss 26477359944295, Testing Loss 25158544232086.406250
Epoch 3500, Training Loss 26477358731955, Testing Loss 25158543116683.707031
Epoch 4000, Training Loss 26477357519618, Testing Loss 25158542001283.949219
Epoch 4500, Training Loss 26477356307282, Testing Loss 25158540885884.972656
Epoch 5000, Training Loss 26477355094948, Testing Loss 25158539770486.781250


tensor([1.0501, 1.0501, 1.0501, 1.0501, 1.0501, 0.0500], requires_grad=True)